In [1]:
!pip install sentence-transformers

import torch
from sentence_transformers import SentenceTransformer, util
import re

intent_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
INTENT_EXAMPLES = {
    "summarize": "summarize this article",
    "search_articles": "find news about a topic",
    "get_topics": "show me the main topics in the news",
    "compare_sources": "compare how different sources cover this",
    "sentiment": "analyze sentiment in this article",
    "similar_articles": "find similar articles",
    "clarification": "clarify what this means",
    "smalltalk": "hello how are you"
}

In [3]:
def classify_intent(query):
  """Return the intent label that best matches the user's query."""

  query_embed = intent_model.encode(query, convert_to_tensor=True)
  best_intent = None
  best_score = -1

  for label, example in INTENT_EXAMPLES.items():
    example_embed = intent_model.encode(example, convert_to_tensor=True)
    score = util.cos_sim(query_embed, example_embed).item()
    if score > best_score:
      best_score = score
      best_intent = label

  return best_intent

In [4]:
def split_(query):
  """
  Automatically split complex queries such as:
  "Summarize this and also show me similar articles"
  """
  parts = re.split(r"\band also\b|\band\b|\bthen\b|\balso\b, query, flags=re.IGNORECASE")
  return [p.strip() for p in parts if p.strip()]

In [5]:
def parse_mult_intent(query):
  """
  For a user query, return a list of (intent, subquery) actions.
  """
  segments = split_query(query)
  commands = []
  for part in segments:
    intent = classify_intent(part)
    commands.append({"intent": intent, "text": part})
  return commands

In [6]:
class ConversationContext:
  def _int_(self):
    self.history = []
    self.last_article_text = None
    self.last_topic = None

def add_turn(self, user_msg, bot_msg):
  self.history.append({"user": user_msg, "bot": bot_msg})
  if len(self.history) > 5:
    self.history.pop(0)

In [7]:
def set_article(self, article_text):
  self.last_article_text = text

def set_topic(self, topic):
  self.last_topic = topic

In [8]:
context = ConversationContext()

In [38]:
def generate_response(intent, data):
  """
  Generate a chatbot response based on the recognized intent.
  This connects your analysis functions with conversastional output.
  """
  if intent == "summarize":
    return f"Here's a concise summary:\n\n{data}"
  elif intent == "search_articles":
    return f"I found these articles related to your topic:\n\n{data}"
  elif intent == "get_topics":
    return f"The main topics are: {data}"
  elif intent == "compare_sources":
    return f"Comparing sources: {data}"
  elif intent == "sentiment":
    return f"Sentiment analysis result: {data}"
  elif intent == "similar_articles":
    return f"Here are similar articles: {data}"
  elif intent == "clarification":
    return f"Clarification for: {data}"
  elif intent == "smalltalk":
    return f"Smalltalk: {data}"
  else:
    return f"I'm not sure how to respond to the intent '{intent}' for: {data}."

In [26]:
def handle_user_query(user_input, analysis_functions):
  """
  analysis_functions = {
    "summarize": function,
    "search_articles": function,

    ...

    }
    """

In [28]:
user_input = [
    "Summarize this article",
    "Find news about a topic",
    "Show me the main topics in the news",
    "Compare how different sources cover this",
    "Analyze sentiment in this article",
    "Find similar articles",
    "Clarify what this means",
    "Hello, how are you?"
]

In [35]:
import re
def split_query(query):
  """
  Split complex user questions into smaller sub-questions.
  Example: "Summarize this and also show me similar articles"
  -> ["Summarize this", "Show me similar articles"]
  """
  parts = re.split(r"\band also\b|\band\b|\bthen\b|\balso\b", query, flags=re.IGNORECASE)

  return [p.strip() for p in parts if p.strip()]

In [39]:
# Define a placeholder for analysis_functions as it's used in the loop
# In a real scenario, this would be populated with actual functions.
analysis_functions = {
    "summarize": lambda text, context: f"Summary of: {text}",
    "search_articles": lambda text, context: f"Searching articles for: {text}",
    "get_topics": lambda text, context: f"Topics for: {text}",
    "compare_sources": lambda text, context: f"Comparing sources for: {text}",
    "sentiment": lambda text, context: f"Sentiment analysis of: {text}",
    "similar_articles": lambda text, context: f"Similar articles to: {text}",
    "clarification": lambda text, context: f"Clarifying: {text}",
    "smalltalk": lambda text, context: f"Smalltalk response for: {text}"
}

all_full_responses = []

# user_input is a list of example queries, so we need to iterate through it
for single_query in user_input:
    parsed_commands = parse_mult_intent(single_query)
    current_query_full_response = []

    for command in parsed_commands:
        intent = command["intent"]
        text = command["text"]

        # Ensure 'context' variable is accessible. It's defined globally in cell 6ka0fgxB5CdD
        # For now, we'll pass the global `context` object.
        # In a more structured application, context might be passed around differently.

        if intent in analysis_functions:
            # Call the dummy analysis function with text and the global context
            result = analysis_functions[intent](text, context)
        else:
            result = f"I don't have a tool for intent '{intent}' for query '{text}' yet."

        response_chunk = generate_response(intent, result)
        current_query_full_response.append(response_chunk)

    all_full_responses.append(f"--- Response for: '{single_query}' ---\n" + "\n\n".join(current_query_full_response))

# Print all accumulated responses
for i, response in enumerate(all_full_responses):
    print(f"--- Processing User Input Example {i+1}: '{user_input[i]}' ---")
    print(response)
    print("-" * 50) # Separator for clarity

--- Processing User Input Example 1: 'Summarize this article' ---
--- Response for: 'Summarize this article' ---
Here's a concise summary:

Summary of: Summarize this article
--------------------------------------------------
--- Processing User Input Example 2: 'Find news about a topic' ---
--- Response for: 'Find news about a topic' ---
I found these articles related to your topic:

Searching articles for: Find news about a topic
--------------------------------------------------
--- Processing User Input Example 3: 'Show me the main topics in the news' ---
--- Response for: 'Show me the main topics in the news' ---
The main topics are: Topics for: Show me the main topics in the news
--------------------------------------------------
--- Processing User Input Example 4: 'Compare how different sources cover this' ---
--- Response for: 'Compare how different sources cover this' ---
Comparing sources: Comparing sources for: Compare how different sources cover this
----------------------